In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import binom
import os, sys, json
sys.path.append('/home/jasper/working/github/')

%matplotlib inline

In [5]:
import koroviev

ImportError: No module named koroviev

In [4]:
sys.path

['',
 '/usr/lib/python2.7',
 '/usr/lib/python2.7/plat-x86_64-linux-gnu',
 '/usr/lib/python2.7/lib-tk',
 '/usr/lib/python2.7/lib-old',
 '/usr/lib/python2.7/lib-dynload',
 '/usr/local/lib/python2.7/dist-packages',
 '/usr/lib/python2.7/dist-packages',
 '/usr/lib/python2.7/dist-packages/PILcompat',
 '/usr/lib/python2.7/dist-packages/gtk-2.0',
 '/usr/lib/pymodules/python2.7',
 '/usr/lib/python2.7/dist-packages/ubuntu-sso-client',
 '/usr/local/lib/python2.7/dist-packages/IPython/extensions',
 '/home/jasper/.ipython',
 '/home/jasper/working/github/']

In [3]:
from koroviev import projections

ImportError: No module named koroviev

In [2]:
from koroviev.koroviev import massey_ranking, projections

ImportError: No module named koroviev.koroviev

In [38]:
kwargs = {'initial_capital': 100,
          'N': 100,
          'binomial_model': binom(1, 0.4)
         }

df = projections.calculate_multiple_iterations_given_criteria(1, kwargs)

df

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,100.0,110.0,121.0,108.9,119.79,107.81,118.59,130.45,117.4,105.66,...,156.12,140.51,126.46,139.11,153.02,168.32,151.49,166.64,149.98,164.98


In [28]:
def convert_results_to_dataframe(M, kwargs):
    '''
    '''
    initial_capital = kwargs['initial_capital']
    N = kwargs['N']
    binomial_model = kwargs['binomial_model']
    iteration_list = []
    for iteration in xrange(M):
        result_array = calculate_returns_after(initial_capital=initial_capital,
            N=N,
            binomial_model=binomial_model)
        iteration_list.append(result_array)

    iteration_array = np.array(iteration_list)
    df = pd.DataFrame(iteration_array) #.transpose())
    return df

In [30]:
def calculate_returns_after(initial_capital, N, binomial_model):
    '''
    '''
    capital_progress = [initial_capital]
    for game in xrange(N):
        capital = capital_progress[-1]
        stake = capital * 0.1
        success = binomial_model.rvs(1)
        if success == 0:
            result = round(stake + capital, 2)
            capital_progress.append(result)
        else:
            result = round((-1 * stake) + capital, 2)
            capital_progress.append(result)

    result_array = np.array(capital_progress)
    return result_array

In [31]:
kwargs = {'initial_capital': 10,
          'N': 1000,
          'binomial_model': binom(1, 0.4)
         }

df = convert_results_to_dataframe(10, kwargs)

In [32]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,1000
0,10.0,9.0,8.1,7.29,8.02,8.82,9.70,8.73,7.86,7.07,...,2.916873e+06,3.208561e+06,3.529417e+06,3.176475e+06,2.858828e+06,2.572945e+06,2.830239e+06,3.113263e+06,2.801937e+06,3.082131e+06
1,10.0,11.0,12.1,13.31,11.98,10.78,9.70,8.73,9.60,10.56,...,1.959253e+08,2.155179e+08,2.370697e+08,2.133627e+08,1.920264e+08,2.112291e+08,2.323520e+08,2.555872e+08,2.811459e+08,3.092605e+08
2,10.0,11.0,12.1,10.89,11.98,13.18,14.50,15.95,14.35,12.91,...,2.153272e+07,1.937945e+07,2.131739e+07,1.918565e+07,2.110422e+07,2.321464e+07,2.089318e+07,1.880386e+07,2.068425e+07,2.275267e+07
3,10.0,11.0,9.9,8.91,9.80,10.78,9.70,10.67,11.74,10.57,...,9.780228e+08,1.075825e+09,1.183408e+09,1.065067e+09,9.585601e+08,1.054416e+09,1.159858e+09,1.275844e+09,1.148259e+09,1.263085e+09
4,10.0,11.0,12.1,13.31,14.64,16.10,17.71,15.94,14.35,15.79,...,6.452832e+06,7.098115e+06,7.807927e+06,7.027134e+06,7.729848e+06,8.502832e+06,9.353116e+06,8.417804e+06,9.259585e+06,1.018554e+07


In [19]:
games = [[0, 3, 3], [1, 4, 21], [1, 5, 5], [2, 0, 13], [2, 3, 13],
         [3, 0, 25], [4, 5, 8], [4, 3, 15], [5, 1, 21], [5, 4, 8]]

In [20]:
def build_games_matrix(games, num_teams):
    """
    - Build the matrix for the specified number of games, and teams.
    """

    M = np.zeros([len(games), num_teams])
    row = 0
    for game in games:
        M[row, game[0]] = 1
        M[row, game[1]] = -1
        row += 1
    return M

def build_outcomes(games):
    """
    This returns a column representing game outcomes
    """
    E = np.zeros([len(games)])
    row = 0
    for game in games:
        E[row] = game[2]
        row += 1
    return E

def solve_lin_eqs(matrix, y_column, kwargs=None):
    """
    - This calculates the solution of the system of linear equations, kwargs
    can specify methodology, it must be a dict.
    """
    solution_output = np.linalg.lstsq(matrix, y_column)

    return solution_output



games_matrix = build_games_matrix(games, 6)
y_column = build_outcomes(games)
solution_output = solve_lin_eqs(games_matrix, y_column)
p_solution = solution_output[0]
sum(map(abs, games_matrix.dot(p_solution) - y_column)) / 10

9.3300000000000018

In [18]:
p_solution

array([-65.37083333,  60.67916667,   8.02916667, -64.57083333,
        20.42916667,  40.80416667])